# Ibsens xmlStrip - *«fjerner buskvekster!»*
Versjon: *0.1.4 Dvergbjerk*

Dato: 09.03.2022

Forfatter: vit.ass. Loke Sjølie

In [1]:
import time
print("Dette programmet tar XML-filer med Ibsenstruktur og gjør dem om til rentekstfiler. Trykk STOPP nå om du ikke vil kjøre.")
print("Har du husket å sette riktige parametre i cellen under?")
time.sleep(3)

Dette programmet tar XML-filer med Ibsenstruktur og gjør dem om til rentekstfiler. Trykk STOPP nå om du ikke vil kjøre.
Har du husket å sette riktige parametre i cellen under?


# Parametre
Under har du parametre for konversjonen.

## his:hisdel
tag_hisdel kontrollerer følgende undertagger: *overwritten*, *overstrike*, *diagonalOverstrike*. Om tag_hisdel = 2 erstattes alle hisdel-taggene med markdown (~), om tag_hisdel = 1 slettes alle helt, og om tag_hisdel er 0 kontrolleres hver type av sin egen verdi. Dvs at GRUPPEN tar prioritet over individuell tagg.

BUG: 2 fungerer ikke... helt. Vet ikke hvorfor. Den finner his:hisdel-taggen(e), men erstatningen inntreffer ikke. Må sannsynligvis bruke en annen metode for å erstatte. Det kan også være fordi disse hisdel-taggene kan være nøstet inni hverandre, kanskje.

In [2]:
# 0 gjør ingenting, 1 sletter taggen med innholdet HELT, 2 beholder innholdet med markdown (~~)
# Gruppekontroll
tag_hisdel = 1

# Individuell kontroll - brukes kun dersom gruppekontroll er 0
# DISSE FUNGERER IKKE ENDA - BRUK GRUPPEKONTROLL 1 ELLER 2, 3 FOR Å DEAKTIVERE. IKKE 0
tag_overwritten = 1
tag_overstrike = 1
tag_diagonaloverstrike = 1

## Filformat
Parametre for å kontrollere hvilket format som skal produseres. Merk at programmet ikke sletter eksisterende filer. Anbefaler å slette alt i output-folderen før det kjøres.

In [3]:
filsuffiks = "_SANERT" # Teksten som skal opptre etter hvert filnavn.
filformat = ".txt" # Formatet på de bearbeidede filene. Anbefaler .txt eller .md.

## Tagger
Parametre for å kontrollere hvilke tagger som skal slettes helt *med* innhold.

In [4]:
#Hvilke tagger skal utraderes MED innhold?
tags_to_erase = ["fw","handnote","teiheader","lb","figure","pb","front","note","anchor","back"] # Front erstattes av rollelisten
#replacemode = "ordinary" # ordinary/regex

# Program
Ikke rør noe under om du ikke vet hva du gjør (med mindre du vet hvordan du spammer undo til det fungerer igjen ;). ).

## Bibliotek

In [5]:
from bs4 import BeautifulSoup # Hent BeautifulSoup-modulen (https://www.crummy.com/software/BeautifulSoup/)
import re # Requests. Brukes i noen regex-funksjoner, gitt
import os # Filsystem for lasting og lagring
import codecs # Dekoder for ASCII
import csv # CSV-leser for ASCII

## Oversett ASCII

In [6]:
file = open('translation.csv') # Egendefinert fil fra DTD. &nbsp, &shy og &tu20 er manuelt oversatt til [mellomrom], [-] og [mellomrom].
# Andre diverse entiteter som manglet er også oversatt.
csvreader = csv.reader(file) # Importer  filen over
asciis = [] # Array for unicode-greier (eks. \u0023)
codes = [] # Array for ting som &nbsp - "html"-koder
for row in csvreader: # For hver rad
    htmlcode = row[0] # Html-tagger (de som er synlige i teksten)
    unicode = row[1] # Unicode (eg \u0023)
    ASCII = codecs.decode(unicode, 'unicode_escape') # Gjør om til ASCII
    asciis.append(ASCII) # Legg till i array
    codes.append(htmlcode)
    print(htmlcode,"=>",ASCII)

&half; => ½
&#x0254; => ɔ
&frac12; => ½
&frac14; => ¼
&frac34; => ¾
&frac18; => ⅛
&frac38; => ⅜
&frac58; => ⅝
&frac78; => ⅞
&sup1; => ¹
&sup2; => ²
&sup3; => ³
&plus; => +
&plusmn; => ±
&equals; => =
&divide; => ÷
&times; => ×
&curren; => ¤
&num; => #
&percnt; => %
&ast; => ∗
&commat; => @
&bsol; => \
&lcub; => {
&horbar; => ―
&verbar; => |
&rcub; => }
&micro; => µ
&ohm; => Ω
&deg; => °
&ordm; => º
&ordf; => ª
&sect; => §
&para; => ¶
&middot; => ·
&larr; => ←
&rarr; => →
&uarr; => ↑
&darr; => ↓
&copy; => ©
&reg; => ®
&trade; => ™
&brvbar; => ¦
&not; => ¬
&sung; => ♩
&excl; => !
&iexcl; => ¡
&quot; => "
&apos; => '
&lpar; => (
&rpar; => )
&comma; => ,
&lowbar; => _
&hyphen; => -
&period; => .
&sol; => /
&colon; => :
&semi; => ;
&quest; => ?
&iquest; => ¿
&laquo; => «
&raquo; => »
&lsquo; => ‘
&rsquo; => ’
&ldquo; => “
&rdquo; => ”
&nbsp; =>  
&shy; => -
&acute; => ´
&breve; => ˘
&caron; => ˇ
&cedil; => ¸
&circ; => ^
&dblac; => ˝
&die; => ¨
&dot; => ˙
&grave; => `
&macr; => ¯
&ogon; => ˛

## Slett tagger vi ikke vil ha

In [7]:
if tag_hisdel == 1:
    tags_to_erase.append("his:hisdel")
elif tag_hisdel == 2:
    print("Alle <his:hisdel>-tagger blir erstattet med markdown: { ~~ [innhold] ~~ }")
    print("Obs, dette fungerer ikke enda. :/")
    # for tag in content.findAll('his:hisdel'):
        # tag.replaceWith("~~ %s ~~" % a.string)
elif tag_hisdel == 0:
    if tag_overwritten == 1:
        tags_to_erase.append("overwritten") 
        # content.findAll("his:hisdel", {"rend" : "overwritten"})
    elif tag_overwritten == 2:
        print("TBI: markdown around overwrittens")
    if tag_overstrike == 1:
        tags_to_erase.append("overstrike") 
        # content.findAll("his:hisdel", {"rend" : "overstrike"})
    elif tag_overstrike == 2:
        print("TBI: markdown around overstrikes")
    if tag_diagonalOverstrike == 1:
        tags_to_erase.append("diagonalOverstrike")
        # content.findAll("his:hisdel", {"rend" : "diagonalOverstrike"})
    elif tag_overstrike == 2:
        print("TBI: markdown around diagonal overstrikes")

## GLOB: *The yeast of thought and mind*
Etablerer filstier, dvs at programmet finner kildefilene.

In [8]:
# GLOB GLOB GOBGALAB
import glob
paths = []
for path in glob.glob('xml_src/**/*.xml', recursive=True):
    paths.append(path)
#paths # Liste over stier til filer i SRC
errors = []
outputs = []
for path in paths:
    path = path.replace('xml_src','xml_output')
    outputs.append(path)
#outputs # Liste over stier til nye filer

### Logg: filstier

In [9]:
i = 0
for path in paths:
    print(i,path)
    i+=1

0 xml_src\brev\B1844-1871ht.xml
1 xml_src\brev\B1871-1879ht.xml
2 xml_src\brev\B1880-1889ht.xml
3 xml_src\brev\B1890-1905ht.xml
4 xml_src\dikt\BlaDigt8191.xml
5 xml_src\dikt\Di01.xml
6 xml_src\dikt\Di41110a.xml
7 xml_src\dikt\Di41113h.xml
8 xml_src\dikt\Di42622.xml
9 xml_src\dikt\Di71.xml
10 xml_src\dikt\Di75.xml
11 xml_src\dikt\Di79.xml
12 xml_src\dikt\Di81928.xml
13 xml_src\dikt\Di82.xml
14 xml_src\dikt\Di86.xml
15 xml_src\dikt\Di91.xml
16 xml_src\dikt\Di96.xml
17 xml_src\dikt\DiFU.xml
18 xml_src\dikt\Diktht.xml
19 xml_src\drama\BE\BE428691_KBK.xml
20 xml_src\drama\BE\BE428691_KBK_gml.xml
21 xml_src\drama\BE\BE428691_KBK_original.xml
22 xml_src\drama\BE\BEht.xml
23 xml_src\drama\BE\BEht_noter.xml
24 xml_src\drama\BE\BEht_original.xml
25 xml_src\drama\BE\BEht_ut.xml
26 xml_src\drama\Br\Br03.xml
27 xml_src\drama\Br\Br05.xml
28 xml_src\drama\Br\Br4262I11.xml
29 xml_src\drama\Br\Br4262I12.xml
30 xml_src\drama\Br\Br66.xml
31 xml_src\drama\Br\Br66a.xml
32 xml_src\drama\Br\Br66b.xml
33 xml_

## Programkjerne
Det er her det skjer.

In [10]:
# ACTION
i = 0
x = 0
y = 0
for thing in paths: # For hver sti (med endestasjon fil) i listen over stier
    print(i+1,thing)
    # Filhåndtering I - Les og last xml-filer inn i minne
    with open(thing, "r", encoding="utf-8") as file: # Åpne stien, les, med encoding UTF8
        # Read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # Combine the lines in the list into a string
        content = "".join(content) # Variabel content får innholdet
    
    # Filhåndtering II - Lag ny sti
    frags = outputs[i].split('\\') # Splitt outputstrengen for sti
    if "xml" in frags[2]: # Hvis filen er 3 ned, altså vi finner xml-filen her (eks. xml_src\dikt\~.xml)
        dirName = frags[0]+"\\"+frags[1] # Dirname blir 2 dyp (xml_output\DIRNAME\)
        metaName = frags[2] # Trenger et navn på filen
    else: # OBS!! OBS!! Hvis filen IKKE er 3 ned ANTAR vi at den er 4 ned, slik som de(n) er i dramafolderen.
        metaName = frags[3] # Navn på filen
        dirName = frags[0]+"\\"+frags[1]+"\\"+frags[2] # Dirname blir 3 dyp (xml_output\drama\DIRNAME\)
        archDir = frags[0]+"\\"+frags[1] # FAILSAFE for DRAMA: Drama er ett hakk lengre ned i hierarkiet enn alt annet. Skap mappe for å ta drama
        #OBS - koden her ANTAR at senteret ALLTID følger samme praksis. Koden må endres om praksis endres.
        if not os.path.exists(archDir): # Hvis subfolder ikke finnes (eks.\drama\BE\)
            os.mkdir(archDir) # Skap den

    metaFolder = dirName+"\\metadata" # Vi forbereder en metadatafolder ett steg under stien vi lagde over
    metaName = metaName[:-4] # Strip . x m l. Bruker ikke outputs[i] fordi den gir hele stien, metaName er fragment
    metaFile = metaFolder+"\\"+metaName+"_METADATA.txt" # Metadatafilen skal få samme filnavn med _metadata-suffiks
    outputs[i] = outputs[i][:-4] # Strip 4 tegn fra slutten av hver streng for å få bort .xml (4. 3X 2M 1L)
    # Create target Directory if don't exist
    if not os.path.exists(dirName): # Hvis folderen ikke eksisterer
        os.mkdir(dirName) # Lag folderen
        os.mkdir(metaFolder) # Legg metadata-folder inne i den igjen
    name = outputs[i]+filsuffiks+filformat # Outputfil får navnet X_SANERT.txt
    # XML til TXT - Det er her vi skal sette inn X FOR EACH for å redigere XML-filene.
    
    content = BeautifulSoup(content, from_encoding="UTF-8") # Content er content som bs4-objekt...
    
    # Hent strukturer vi skal beholde
    try:
        titlestatement = content.find("titlestmt") # Finn title statement
        titlestatement.extract() # Hent (og fjern) title statement-delen
    except:
        print("> Ingen title statement.")

    try:
        filedescription = content.find("filedesc") # Finn filbeskrivelsen
        filedescription.extract() # Hent (og fjern) filbeskrivelsen
    except:
        print("> Ingen filbeskrivelse.")
    try:
        castlist = content.find("his:hiscastlist") # Finn rollelisten
        castlist.extract() # Hent (og fjern) rollelisten
        print("Hentet rollelisten")
        checkList = 1
    except:
        print("> Ingen rolleliste.")
        checkList = 0
        
    if checkList == 1: # Kan gjøre try, men...
        target = content.body # Mål: Body-taggen
        castlist.append(" ") # Legg til et mellomrom helt på slutten for sikkerhets skyld.
        target.insert_before(castlist) # Sett inn castlist foran target
        #target = content.front # Mål (<front>)
        #target.replace_with(castlist) # ERSTATT målet med castlist-taggen
        
    # Slett tagger vi ikke skal beholde
    if tag_hisdel == 2:
        z = 0
        try:
            for tag in content.findAll('his:hisdel'):
                tag.replaceWith(" ~~ %s ~~ " % a.string)
                z += 1
            if z > 0:
                print(">Erstattet",z,"hisdel-tagger med markdown-tagger.")
            else:
                print(">his:hisdel-tagger er ikke 0, men 0 ble fjernet...? Hva.")
        except:
            print("> Ingen hisdel-tagger å endre.")
            
            
    try: # Her er kode for å endre & bevare replikker
        speaker = content.findAll('speaker') # Finn alle replikker
        for talker in speaker: # For hver av disse
            tag = content.new_tag("dummytag") # Lag en ny, tom tagg
            tag.string = (talker.string) # Inkonsekvent punktum- og mellomromsføring fører til at vi ikke kan legge ->
                                        # mellomrom eller punktum etter denne.
            talker.insert_before(tag) # Sett elementet inn før replikken
            talker.decompose() # Ødelegg kildeelementet for å hindre dobbelt opp
        print("> Fikset replikker.")
    except:
        print("> Fant ingen replikker.")
    try:
        for tag in tags_to_erase: # For hver tagg
            to_remove = content.find_all(tag) # Finn alle instanser av taggen
            for remove in to_remove: # For hver av disse
                remove.decompose() # Fjern taggen og innholdet helt
        # Neste 3 linjer fjerner tilfeldige tall i output-filene. Disse er SPESIFIKT tall (subtype:no)
        supralinears = content.findAll(attrs={"rend": "supralinear", "subtype":"no"}) # Finn ^x-notasjon
        for no in supralinears: # For hver av disse
            no.decompose() # Slett dem helt
        infralinears = content.findAll(attrs={"rend": "infralinear", "subtype":"no"}) # Finn ^x-notasjon..
        for no in infralinears: # For hver av disse
            no.decompose() # Slett dem helt
        print("> Fjernet uønskede tagger.")
    except:
        print("> Fant ingen uønskede tagger.")
    content = str(content) # CONTENT ER NÅ STRING - STRINGMETODER FUNGERER, BS4-METODER FUNGERER IKKE
    z = 0 # Tell fra 0 i z
    for code in codes: # For hver kode
        if code in content: # Hvis koden finnes i teksten
            content = content.replace(code,asciis[z]) # Erstatt koden med ascii
            #print('Erstattet',code,'med',asciis[z]) # Print bekreftelse
        z+=1 # Neste kode/ascii etter å ha sjekket hver kode
    print("> Erstattet feltkoder med ASCII.")
    
    content = re.sub('<[^<]+>', "", content) # Slett alle resterende tagger UTEN å røre innholdet
    content = re.sub('\s+',' ', content) # Korriger alle instanser av multimellomrom til enkeltmellomrom
    content = content.replace("&typHyp; ","")
    content = content.replace("&typHyp;","")
    #if replacemode == "ordinary":
    content = content.replace("\n"," ") # Slett newline og erstatt med mellomrom 
    content = content.replace("'“'","")
    content = content.replace("&dash;","-")
    content = content.replace("&"," & ")
    content = content.replace("& &"," & ")
    content = content.replace(". &",".")
    #content = content.replace(" > ","")
    content = content.replace("'\"'", "")
    content = content.replace(" < ","")
        #content = content.replace("&typHyp;","") # Slett disse etterpå.
    #elif replacemode == "regex": # Ikke bruk denne tror jeg egentlig
    #    content = content.replace("\n"," ")
    #    content = re.sub("'“'", "", content) # Korriger enkelte instanser av '“'
    #    content = re.sub("'\"'", "", content) # Korriger enkelte instanser av '"'...
    #    content = re.sub("&dash;","–",content) # Korriger &dash; fordi den ikke vil korrigeres gjennom translation.csv??
    #    content = re.sub("&", " & ", content) # Her finner vi alle ampersander og legger mellomrom rundt. Disse slettes etterpå.
    #    content = re.sub('\s+',' ', content) # Korriger alle instanser av multimellomrom til enkeltmellomrom
    #    content = re.sub("& &", "&", content) # Korriger dobbeltampersander etter mellomrom er fjernet.
    #    content = re.sub(". &", ".", content) # Starter da ikke en setning med ampersand-tegn, vel...?
    #    content = re.sub(" < ","",content) # Fjerner " < " fra starten av dokumentet og/eller andre steder.
    content = re.sub('\s+',' ', content) # Korriger alle instanser av multimellomrom til enkeltmellomrom IGJEN.
    # For sikkerhets skyld.
    
    if tag_hisdel == 1: # Hvis vi sletter hisdel-taggen med innhold
        #content = re.sub(" . ", " ", content) # NEI!
        content = content.replace(" . ", " ") # JA!
        content = content.replace(" < ", "") # JA!
    print("> Formatert.")
    # Filhåndtering III - Skriv til disk
    try: # Prøv å skrive til disk
        f = open(name, "w", encoding="utf-8") # Ikke glem encoding="utf-8", sånn som jeg gjorde.
        f.write(content) # Skriv innholdet i content-variabelen
        f.close()
        print(">> Opprettet",name,"i",dirName)
        x+=1
    except: # Hvis vi møter på feil, rapporter kilden
        print("XXX FILFEIL!",thing)
        f = open(name+"_ERROR", "w", encoding="utf-8")
        f.write("XXX Filfeil!")
        f.close()
        errors.append(name)
        y+=1
    metadata = str(filedescription)+str(titlestatement)
    try: # Prøv å skrive metadata! til disk
        f = open(metaFile, "w", encoding="utf-8") # Ikke glem encoding="utf-8", sånn som jeg gjorde.
        f.write(metadata) # Skriv innholdet i metadata-variabelen
        f.close()
        print(">> Metadata opprettet:",metaFile)
        x+=1
    except: # Hvis vi møter på feil, rapporter kilden
        print("YYY FILFEIL!",thing)
        f = open(metaFile+"_ERROR", "w", encoding="utf-8")
        f.write("YYY Metadata filfeil!")
        f.close()
        errors.append(name)
        y+=1
    i+=1
    #if i > 70:
    #    raise KeyboardInterrupt
    print("\n")
print("\nLog:",x,"files parsed to text.",y,"files failing to parse.")
if y > 0:
    face = ":("
else:
    face = ":)"

1 xml_src\brev\B1844-1871ht.xml


C:\Users\lokes\Anaconda3\lib\site-packages\bs4\__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


> Ingen rolleliste.
> Fikset replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\brev\B1844-1871ht_SANERT.txt i xml_output\brev
>> Metadata opprettet: xml_output\brev\metadata\B1844-1871ht_METADATA.txt


2 xml_src\brev\B1871-1879ht.xml
> Ingen rolleliste.
> Fikset replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\brev\B1871-1879ht_SANERT.txt i xml_output\brev
>> Metadata opprettet: xml_output\brev\metadata\B1871-1879ht_METADATA.txt


3 xml_src\brev\B1880-1889ht.xml
Hentet rollelisten
> Fant ingen replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\brev\B1880-1889ht_SANERT.txt i xml_output\brev
>> Metadata opprettet: xml_output\brev\metadata\B1880-1889ht_METADATA.txt


4 xml_src\brev\B1890-1905ht.xml
> Ingen rolleliste.
> Fikset replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Oppre

Hentet rollelisten
> Fant ingen replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\Br\Br4262I12_SANERT.txt i xml_output\drama\Br
>> Metadata opprettet: xml_output\drama\Br\metadata\Br4262I12_METADATA.txt


31 xml_src\drama\Br\Br66.xml
Hentet rollelisten
> Fant ingen replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\Br\Br66_SANERT.txt i xml_output\drama\Br
>> Metadata opprettet: xml_output\drama\Br\metadata\Br66_METADATA.txt


32 xml_src\drama\Br\Br66a.xml
Hentet rollelisten
> Fikset replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\Br\Br66a_SANERT.txt i xml_output\drama\Br
>> Metadata opprettet: xml_output\drama\Br\metadata\Br66a_METADATA.txt


33 xml_src\drama\Br\Br66b.xml
Hentet rollelisten
> Fant ingen replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprett

> Formatert.
>> Opprettet xml_output\drama\C1\C14936_1-17_19-47_SANERT.txt i xml_output\drama\C1
>> Metadata opprettet: xml_output\drama\C1\metadata\C14936_1-17_19-47_METADATA.txt


59 xml_src\drama\C1\C14936_18r.xml
> Ingen rolleliste.
> Fikset replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\C1\C14936_18r_SANERT.txt i xml_output\drama\C1
>> Metadata opprettet: xml_output\drama\C1\metadata\C14936_18r_METADATA.txt


60 xml_src\drama\C1\C14936_18v.xml
> Ingen rolleliste.
> Fikset replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\C1\C14936_18v_SANERT.txt i xml_output\drama\C1
>> Metadata opprettet: xml_output\drama\C1\metadata\C14936_18v_METADATA.txt


61 xml_src\drama\C1\C14936_NBO.xml
Hentet rollelisten
> Fant ingen replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\C1\C14936_NBO_SANERT.txt i xml_

...

> Ingen rolleliste.
> Fant ingen replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\O1\O1344_FruKirsten_SANERT.txt i xml_output\drama\O1
>> Metadata opprettet: xml_output\drama\O1\metadata\O1344_FruKirsten_METADATA.txt


367 xml_src\drama\O1\O1344_Hemming.xml
> Ingen rolleliste.
> Fant ingen replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\O1\O1344_Hemming_SANERT.txt i xml_output\drama\O1
>> Metadata opprettet: xml_output\drama\O1\metadata\O1344_Hemming_METADATA.txt


368 xml_src\drama\O1\O1344_Ingeborg.xml
> Ingen rolleliste.
> Fant ingen replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\O1\O1344_Ingeborg_SANERT.txt i xml_output\drama\O1
>> Metadata opprettet: xml_output\drama\O1\metadata\O1344_Ingeborg_METADATA.txt


369 xml_src\drama\O1\O1344_MaendKvinder1.xml
> Ingen rolleliste.
> Fant ingen r

Hentet rollelisten
> Fikset replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\PG\PG81_SANERT.txt i xml_output\drama\PG
>> Metadata opprettet: xml_output\drama\PG\metadata\PG81_METADATA.txt


395 xml_src\drama\PG\PG81940.xml
> Ingen rolleliste.
> Fikset replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\PG\PG81940_SANERT.txt i xml_output\drama\PG
>> Metadata opprettet: xml_output\drama\PG\metadata\PG81940_METADATA.txt


396 xml_src\drama\PG\PG85.xml
Hentet rollelisten
> Fikset replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\PG\PG85_SANERT.txt i xml_output\drama\PG
>> Metadata opprettet: xml_output\drama\PG\metadata\PG85_METADATA.txt


397 xml_src\drama\PG\PG86.xml
Hentet rollelisten
> Fikset replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output

> Ingen rolleliste.
> Fant ingen replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\Sa\Sa872r_JohannesBirk_SANERT.txt i xml_output\drama\Sa
>> Metadata opprettet: xml_output\drama\Sa\metadata\Sa872r_JohannesBirk_METADATA.txt


425 xml_src\drama\Sa\Sa872r_Jorgen.xml
> Ingen rolleliste.
> Fikset replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\Sa\Sa872r_Jorgen_SANERT.txt i xml_output\drama\Sa
>> Metadata opprettet: xml_output\drama\Sa\metadata\Sa872r_Jorgen_METADATA.txt


426 xml_src\drama\Sa\Sa872r_Juliane.xml
> Ingen rolleliste.
> Fant ingen replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\Sa\Sa872r_Juliane_SANERT.txt i xml_output\drama\Sa
>> Metadata opprettet: xml_output\drama\Sa\metadata\Sa872r_Juliane_METADATA.txt


427 xml_src\drama\Sa\Sa872r_LidenKarin.xml
> Ingen rolleliste.
> Fikset repli

Hentet rollelisten
> Fant ingen replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\SS\SS41112_NBO_SANERT.txt i xml_output\drama\SS
>> Metadata opprettet: xml_output\drama\SS\metadata\SS41112_NBO_METADATA.txt


453 xml_src\drama\SS\SS77.xml
Hentet rollelisten
> Fant ingen replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\SS\SS77_SANERT.txt i xml_output\drama\SS
>> Metadata opprettet: xml_output\drama\SS\metadata\SS77_METADATA.txt


454 xml_src\drama\SS\SS77a.xml
Hentet rollelisten
> Fant ingen replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\SS\SS77a_SANERT.txt i xml_output\drama\SS
>> Metadata opprettet: xml_output\drama\SS\metadata\SS77a_METADATA.txt


455 xml_src\drama\SS\SS81469_1-2.xml
> Ingen rolleliste.
> Fikset replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formate

Hentet rollelisten
> Fant ingen replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\UF\UFht_SANERT.txt i xml_output\drama\UF
>> Metadata opprettet: xml_output\drama\UF\metadata\UFht_METADATA.txt


481 xml_src\drama\UF\UFht_noter.xml
> Ingen rolleliste.
> Fikset replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\UF\UFht_noter_SANERT.txt i xml_output\drama\UF
>> Metadata opprettet: xml_output\drama\UF\metadata\UFht_noter_METADATA.txt


482 xml_src\drama\Vi\Vi41115a.xml
> Ingen rolleliste.
> Fant ingen replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII.
> Formatert.
>> Opprettet xml_output\drama\Vi\Vi41115a_SANERT.txt i xml_output\drama\Vi
>> Metadata opprettet: xml_output\drama\Vi\metadata\Vi41115a_METADATA.txt


483 xml_src\drama\Vi\Vi41115b.xml
Hentet rollelisten
> Fant ingen replikker.
> Fjernet uønskede tagger.
> Erstattet feltkoder med ASCII

## Logg

In [11]:
print("Gjorde om",i+1,"dokumenter til rentekst fra",len(paths)+1,"XML-filer. Vi fikk",y,"encoding-feil.",face)
if errors:
    errors

Gjorde om 494 dokumenter til rentekst fra 494 XML-filer. Vi fikk 0 encoding-feil. :)


~~UnicodeEncodeError: 'charmap' codec can't encode character '\u203b' in position 1180: character maps to undefined. Denne karakteren her: ※ (Fixed)~~

~~Ok, fra 3 feil til 1 etter å ha konvertert "&ergo;" til "ɔ"!~~
Header
FIX: Glemte å manuelt sette output til encoding="utf-8"... men nå skriver alle filer!

~~TODO: Mange korreksjoner og sånne ting som gjemmer seg blant teksten i tags.~~

~~TODO: De variablene under må dekonstrueres og deler av dem må stappes inn igjen inn i en ny fil under metadata~~.

# Ferdig!
De rene tekstfilene skal nå finnes under .../xml_output/*, samt metadata i tilhørende mappe. Med mindre du fikk feil i scriptet over. Eller om noe annet gikk galt.

### TODO
Ja, det er jo selvfølgelig HTML-kommentarer (< !-- -->) i *noen* av disse.. de må fjernes.

___

# Testing
Ting under denne overskriften er bare å se bort fra med mindre du skal teste nye greier eller andre løsninger. Obs - dette fungerer kun etter å ha kjørt hele koden en gang.

Prøver nå å finne ut hvorfor enkelt-ier (" i ") forsvinner i prod...

In [12]:
raise KeyboardInterrupt # Stooooopppp.

KeyboardInterrupt: 

In [13]:
item = 19 # 

thing = paths[item]
with open(thing, "r", encoding="utf-8") as file: # Åpne stien, les, med encoding UTF8
    # Read each line in the file, readlines() returns a list of lines
    content = file.readlines()
    # Combine the lines in the list into a string
    content = "".join(content) # Variabel content får innholdet
content = BeautifulSoup(content, from_encoding="UTF-8") # Content er content som bs4-objekt...

In [14]:
outputs[item] = outputs[item] + ".xml"

In [15]:
# Filhåndtering II - Lag ny sti
frags = outputs[item].split('\\') # Splitt outputstrengen for sti
if "xml" in frags[2]: # Hvis filen er 3 ned, altså vi finner xml-filen her (eks. xml_src\dikt\~.xml)
    dirName = frags[0]+"\\"+frags[1] # Dirname blir 2 dyp (xml_output\DIRNAME\)
    metaName = frags[2] # Trenger et navn på filen
else: # OBS!! OBS!! Hvis filen IKKE er 3 ned ANTAR vi at den er 4 ned, slik som de(n) er i dramafolderen.
    metaName = frags[3] # Navn på filen
    dirName = frags[0]+"\\"+frags[1]+"\\"+frags[2] # Dirname blir 3 dyp (xml_output\drama\DIRNAME\)
    archDir = frags[0]+"\\"+frags[1] # FAILSAFE for DRAMA: Drama er ett hakk lengre ned i hierarkiet enn alt annet. Skap mappe for å ta drama
    #OBS - koden her ANTAR at senteret ALLTID følger samme praksis. Koden må endres om praksis endres.
    if not os.path.exists(archDir): # Hvis subfolder ikke finnes (eks.\drama\BE\)
        os.mkdir(archDir) # Skap den

metaFolder = dirName+"\\metadata" # Vi forbereder en metadatafolder ett steg under stien vi lagde over
metaName = metaName[:-4] # Strip . x m l. Bruker ikke outputs[i] fordi den gir hele stien, metaName er fragment
metaFile = metaFolder+"\\"+metaName+"_METADATA.txt" # Metadatafilen skal få samme filnavn med _metadata-suffiks
outputs[item] = outputs[item][:-4] # Strip 4 tegn fra slutten av hver streng for å få bort .xml (4. 3X 2M 1L)
# Create target Directory if don't exist
if not os.path.exists(dirName): # Hvis folderen ikke eksisterer
    os.mkdir(dirName) # Lag folderen
    os.mkdir(metaFolder) # Legg metadata-folder inne i den igjen
name = outputs[item]+filsuffiks+filformat # Outputfil får navnet X_SANERT.txt
# XML til TXT - Det er her vi skal sette inn X FOR EACH for å redigere XML-filene.

In [16]:
# Hent strukturer vi skal beholde
try:
    titlestatement = content.find("titlestmt") # Finn title statement
    titlestatement.extract() # Hent (og fjern) title statement-delen
except:
    print("> Ingen title statement.")

try:
    filedescription = content.find("filedesc") # Finn filbeskrivelsen
    filedescription.extract() # Hent (og fjern) filbeskrivelsen
except:
    print("> Ingen filbeskrivelse.")
try:
    castlist = content.find("his:hiscastlist") # Finn rollelisten
    castlist.extract() # Hent (og fjern) rollelisten
    print("Hentet rollelisten")
    checkList = 1
except:
    print("> Ingen rolleliste.")
    checkList = 0

if checkList == 1: # Kan gjøre try, men...
    target = content.body # Mål: Body-taggen
    castlist.append(" ") # Legg til et mellomrom helt på slutten for sikkerhets skyld.
    target.insert_before(castlist) # Sett inn castlist foran target
    #target = content.front # Mål (<front>)
    #target.replace_with(castlist) # ERSTATT målet med castlist-taggen

# Slett tagger vi ikke skal beholde
if tag_hisdel == 2:
    z = 0
    try:
        for tag in content.findAll('his:hisdel'):
            tag.replaceWith(" ~~ %s ~~ " % a.string)
            z += 1
        if z > 0:
            print(">Erstattet",z,"hisdel-tagger med markdown-tagger.")
        else:
            print(">his:hisdel-tagger er ikke 0, men 0 ble fjernet...? Hva.")
    except:
        print("> Ingen hisdel-tagger å endre.")

> Ingen rolleliste.


In [17]:
try: # Her er kode for å endre & bevare replikker
    speaker = content.findAll('speaker') # Finn alle replikker
    for talker in speaker: # For hver av disse
        tag = content.new_tag("dummytag") # Lag en ny, tom tagg
        tag.string = (talker.string) # Inkonsekvent punktum- og mellomromsføring fører til at vi ikke kan legge ->
                                    # mellomrom eller punktum etter denne.
        talker.insert_before(tag) # Sett elementet inn før replikken
        talker.decompose() # Ødelegg kildeelementet for å hindre dobbelt opp
    print("> Fikset replikker.")
except:
    print("> Fant ingen replikker.")

> Fikset replikker.


In [18]:
content

<?xml version="1.0" encoding="UTF-8"?><!DOCTYPE TEI>
<html><body><p>
%HIS_entiteter;
]&gt;
<?xml-model href="http://www.edd.uio.no/ibsen/schema/tei_his.rnc" type="application/relax-ng-compact-syntax"?>
<tei xmlns="http://www.tei-c.org/ns/1.0" xmlns:his="http://www.example.org/ns/HIS">
<teiheader xml:lang="nob">

<encodingdesc>
<projectdesc><p>&amp;projectDesc-grt-ms;</p></projectdesc>
<editorialdecl>
<his:typography>
<list>
Episk Brand - KBK NKS 2869, 4°, 1
<item>Manuskriptet er satt sammen av flere arbeidsmanuskripter; dels kladder med mange rettelser, dels renskrift med få rettelser. Materialet er egenhendig og det er blitt brukt flere blekkfarger (se håndlista). Vår beskrivelse av typografien blir derfor svært generell siden materialets typografi varierer, og siden oppsettet er preget av at dette er arbeidsmanuskript.</item>
<item>skriftgruppe: latinsk håndskrift, enkelte gotiske innslag, majuskler og minuskler, høyrehellende</item>
<item>teksten:
<list>
<item>overskrifter: midtstil

In [ ]:
try:
    for tag in tags_to_erase: # For hver tagg
        to_remove = content.find_all(tag) # Finn alle instanser av taggen
        for remove in to_remove: # For hver av disse
            remove.decompose() # Fjern taggen og innholdet helt
    print("> Fjernet uønskede tagger.")
except:
    print("> Fant ingen uønskede tagger.")
content = str(content) # CONTENT ER NÅ STRING - STRINGMETODER FUNGERER, BS4-METODER FUNGERER IKKE
z = 0 # Tell fra 0 i z
for code in codes: # For hver kode
    if code in content: # Hvis koden finnes i teksten
        content = content.replace(code,asciis[z]) # Erstatt koden med ascii
        #print('Erstattet',code,'med',asciis[z]) # Print bekreftelse
    z+=1 # Neste kode/ascii etter å ha sjekket hver kode
print("> Erstattet feltkoder med ASCII.")

In [ ]:
content = re.sub('<[^<]+>', "", content) # Slett alle resterende tagger UTEN å røre innholdet
#content = content.replace("\n"," ") # Slett newline og erstatt med mellomrom 
    #content = re.sub('\s+',' ', content) # Korriger alle instanser av multimellomrom til enkeltmellomrom
content = content.replace("&typHyp; ","")
content = content.replace("&typHyp;","")
#if replacemode == "ordinary":

content = content.replace("'“'","")
content = content.replace("&dash;","-")
content = content.replace("&"," & ")
content = content.replace("& &"," & ")
content = content.replace(". &",".")
content = content.replace("'\"'", "")
content = content.replace(" < ","")

content = re.sub('\s+',' ', content) # Korriger alle instanser av multimellomrom til enkeltmellomrom IGJEN.
# For sikkerhets skyld.

if tag_hisdel == 1: # Hvis vi sletter hisdel-taggen med innhold
    content = content.replace(" . ", " ") # JA!
    content = content.replace(" < ", "") # JA!
print("> Formatert.")

In [ ]:
content

In [ ]:
# Filhåndtering III - Skriv til disk
try: # Prøv å skrive til disk
    f = open(name, "w", encoding="utf-8") # Ikke glem encoding="utf-8", sånn som jeg gjorde.
    f.write(content) # Skriv innholdet i content-variabelen
    f.close()
    print(">> Opprettet",name,"i",dirName)
    x+=1
except: # Hvis vi møter på feil, rapporter kilden
    print("XXX FILFEIL!",thing)
    f = open(name+"_ERROR", "w", encoding="utf-8")
    f.write("XXX Filfeil!")
    f.close()
    errors.append(name)
    y+=1
metadata = str(filedescription)+str(titlestatement)
try: # Prøv å skrive metadata! til disk
    f = open(metaFile, "w", encoding="utf-8") # Ikke glem encoding="utf-8", sånn som jeg gjorde.
    f.write(metadata) # Skriv innholdet i metadata-variabelen
    f.close()
    print(">> Metadata opprettet:",metaFile)
    x+=1
except: # Hvis vi møter på feil, rapporter kilden
    print("YYY FILFEIL!",thing)
    f = open(metaFile+"_ERROR", "w", encoding="utf-8")
    f.write("YYY Metadata filfeil!")
    f.close()
    errors.append(name)
    y+=1
#if i > 70:
#    raise KeyboardInterrupt
print("\nTesten ble kjørt.")

In [ ]:
content